In [1]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

fill_value = -32767.0

nc_file_1 = nc.Dataset('AQUA_MODIS-Copy1.20040101_20041231.L3m.YR.SST.sst.4km.nc', 'r')
sst_data_1 = np.array(nc_file_1.variables['sst'][:])
sst_data_1 = np.where(sst_data_1 == fill_value, np.nan, sst_data_1)
latitudes = np.array(nc_file_1.variables['lat'][:])
longitudes = np.array(nc_file_1.variables['lon'][:])
nc_file_1.close()

nc_file_2 = nc.Dataset('AQUA_MODIS-Copy1.20130101_20131231.L3m.YR.SST.sst.4km.nc', 'r')
sst_data_2 = np.array(nc_file_2.variables['sst'][:])
sst_data_2 = np.where(sst_data_2 == fill_value, np.nan, sst_data_2)
nc_file_2.close()

nc_file_3 = nc.Dataset('AQUA_MODIS-Copy1.20230101_20231231.L3m.YR.SST.sst.4km.nc', 'r')
sst_data_3 = np.array(nc_file_3.variables['sst'][:])
sst_data_3 = np.where(sst_data_3 == fill_value, np.nan, sst_data_3)
nc_file_3.close()

# Create latitude and longitude grids
lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()

sst_data_1 = sst_data_1.flatten()
sst_data_2 = sst_data_2.flatten()
sst_data_3 = sst_data_3.flatten()

filtered_df = pd.DataFrame({
    'Latitude': lat_flat,
    'Longitude': lon_flat,
    '2004 Data': sst_data_1,
    '2013 Data': sst_data_2,
    '2023 Data': sst_data_3
})

# Filter by latitude and longitude ranges
lat_range = (-30, 30)
lon_range = (30, 120)
filtered_df = filtered_df[(filtered_df['Latitude'] >= lat_range[0]) & (filtered_df['Latitude'] <= lat_range[1]) & 
                 (filtered_df['Longitude'] >= lon_range[0]) & (filtered_df['Longitude'] <= lon_range[1])]

filtered_df = filtered_df.dropna(subset=['2004 Data', '2013 Data', '2023 Data'], how='all')  # Drop rows where all SST data is NaN

filtered_df = filtered_df.ffill().bfill()

years = np.array([2004, 2013, 2023])
X = np.vstack([np.column_stack((filtered_df[['Latitude', 'Longitude']].values, np.full(filtered_df.shape[0], year))) for year in years])
y = np.hstack([filtered_df[f'{year} Data'].values for year in years])


degree = 3
poly = PolynomialFeatures(degree)
model = make_pipeline(poly, Ridge(alpha=1.0))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

C:\Users\ypava\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.85497e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('ridge', Ridge())])

In [6]:
ridge_model = model.named_steps['ridge']
poly_features = model.named_steps['polynomialfeatures']

coefs = ridge_model.coef_
intercept = ridge_model.intercept_

terms = ["1"] + [f"x^{i}" for i in range(1, degree + 1)]
equation = " + ".join([f"{coef}*{term}" for coef, term in zip(coefs, terms)])
polynomial_function = f"{intercept} + {equation}"

print("Polynomial function: ", polynomial_function)

Polynomial function:  2878.8977358133147 + 0.0*1 + 17.935893495453353*x^1 + -99.46305293169219*x^2 + -1.1621613432545299e-05*x^3


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Any NaN in X_test:", np.any(np.isnan(X_test)))
print("Any NaN in y_test:", np.any(np.isnan(y_test)))

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

sst_predictions = y_pred[:]

for i in range(0,len(X_test),len(X_test)//100) :
    print(X_test[i],"  ",sst_predictions[i],"  ",y_test[i])


Any NaN in X_test: False
Any NaN in y_test: False
Mean Squared Error: 1.2237713801438366
R-squared Score: 0.80848835995038
[  23.22916603  118.68752289 2004.        ]    25.900160142003188    24.055
[   4.06249475   74.77083588 2013.        ]    29.159259586747794    29.47
[ -24.47916985   33.18750381 2023.        ]    26.70259326791438    35.17
[   9.68749523   72.14583588 2023.        ]    29.210819611689658    29.609999
[  -3.02083325   67.68750763 2004.        ]    28.621423739206875    29.33
[ -18.72916985   79.93752289 2023.        ]    25.82078534617085    25.529999
[   9.39583111   58.10417557 2023.        ]    28.807376487729925    28.314999
[  -8.18750477   72.43750763 2013.        ]    28.237282242151196    28.224998
[  12.77083111   80.64583588 2004.        ]    28.73026475718052    28.42
[  15.56249523   60.02083206 2023.        ]    28.562692504482584    27.685
[  -6.77083349   99.47917938 2004.        ]    28.956101452274197    29.025
[ -18.72916985   52.93750381 2023.  

In [74]:
fill_value=-32767.0
nc_file_4 = nc.Dataset('AQUA_MODIS-Copy1.20020101_20021231.L3m.YR.SST.sst.4km (3).nc', 'r')
sst_data_4 = np.array(nc_file_4.variables['sst'][:])
sst_data_4 = np.where(sst_data_4 == fill_value, np.nan, sst_data_4)
nc_file_4.close()
sst_data_4=sst_data_4.reshape(-1)
print(sst_data_4.shape)
lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()


test_df = pd.DataFrame({
    'Latitude': lat_flat,
    'Longitude': lon_flat,
    'SST Data': sst_data_4
})

test_df=test_df.dropna()
print( np.any(np.isnan(test_df['SST Data'])))

(37324800,)
False
